In [29]:
import pandas as pd
import ast
from pandas import json_normalize
from Bio import SeqIO
import csv
import os
from ete3 import NCBITaxa
import numpy as np
import sys
import json
import traceback
import csv
import glob
import re
ncbi = NCBITaxa()

## GC content table

In [2]:
df_gc = pd.read_csv('files/output/all__genomes_GC_content.tsv', sep='\t')
df_gc.reset_index(drop=True, inplace=True)

df_gc['Genome_name'] = df_gc['Genome_name'].str.split('.').str[0]
df_gc.rename(columns={'Genome_name': 'Genome_ID'}, inplace=True)

## Parsing of MGnify, JGI and HMP metadata

In [3]:
# Format GTDB taxonomy into the dictionary
def format_taxonomy(value):
    replacements = {'d': 'Domain', 'p': 'Phylum', 'c': 'Class', 'o': 'Order', 'f': 'Family', 'g': 'Genus', 's': 'Species'}
    parts = value.split(';')

    formatted_taxonomy = {}
    for part in parts:
        if '__' in part:
            letter, taxonomy = part.split('__')
            if letter in replacements:
                formatted_part = {replacements[letter]: taxonomy}
            else:
                continue  # Skip unrecognized letters
        else:
            formatted_part = {'Unknown': part}
        formatted_taxonomy.update(formatted_part)

    return formatted_taxonomy

# Extract taxonomy from dictionaries to columns
def extract_taxon_info(row, key):
    if row['Taxon'] is None:
        return None
    return row['Taxon'][key] if key in row['Taxon'] else None

def taxid2names(taxid):
    """
    Output example: 2: {'Bacteria', 1760: 'Actinomycetes'}
    """
    taxid = int(taxid)
    tax_name = ncbi.get_taxid_translator([taxid])[taxid]
    lineage = ncbi.get_taxid_translator(ncbi.get_lineage(taxid))
    x = list(lineage.values())
    return lineage

def rank2taxid(gene_taxid):
    """
    Output example: {'superkingdom': 2, 'class': 1760}
    """
    gene_lineage = ncbi.get_lineage(gene_taxid)
    gene_lineage2ranks = ncbi.get_rank(gene_lineage)
    gene_ranks2lineage = dict((rank, taxid) for (taxid, rank) in gene_lineage2ranks.items())
    return gene_ranks2lineage

In [4]:
def compute_quality(row):
    """
    Assign the quality level based on the N50 and the number of fasta records within 1 fna
    """
    
    if 'Isolate' in row['Genome_type']:
        if row['Count'] > 5 or row['N50'] < 2500000:
            return 'Isolate (fragmented)'
        else:
            return 'Isolate (complete)'
    else:
        return 'MAG'

#### Sample classification table

In [30]:
# Upload site classification table (old version)
df_class = pd.read_csv('files/input/specimen_classifications/specimen_classification.tsv', delimiter='\t')
df_class['Original_HMP'] = df_class['Original_HMP'].str.lower()
df_class['Original_JGI'] = df_class['Original_JGI'].str.lower()
df_class['Original_HMP'] = df_class['Original_HMP'].str.split(',')
df_class['Original_JGI'] = df_class['Original_JGI'].str.split(',')

In [31]:
df_class.head()

,Specimen,Category,Specimen_Type,Original_HMP,Original_JGI
0,Gut_unclassified,Gut,Gut_unclassified,[none],"[digestive system/host-associated, digestive s..."
1,Gut_colon,Gut,Colon,"[feces, stool]","[digestive system/huma fecal, digestive system..."
2,Gut_bile_duct,Gut,Bile_duct,[none],[digestive system/human bile duct]
3,Oral_unclassified,Oral,Oral_unclassified,[none],[digestive system/human oral]
4,Oral_plaque,Oral,Plaque,"[attached keratinized gingiva, subgingival pla...",[digestive system/fossilized dental plaque]
5,Oral_mucosa,Oral,Mucosa,[buccal mucosa],[none]
6,Oral_saliva,Oral,Saliva,"[portion of saliva, saliva]",[none]
7,Oral_cavity,Oral,Oral_cavity,"[hard palate, tongue dorsum]",[none]
8,Respiratory_unclassified,Respiratory,Respiratory_unclassified,[none],[respiratory system/human]
9,Respiratory_upper,Respiratory,Upper_respiratory,"[anterior nares, palatine tonsils, throat]",[none]


### Mgnify

#### Parse MGnify gut

In [8]:
# Read MGnify gut metadata file
df_mg_g = pd.read_csv('files/input/mgnify_meta/genomes-all_metadata_gut.tsv', delimiter='\t')
df_mg_g_counts = pd.read_csv('files/output/mgnify_meta/mgnify_gut_fasta_counts.tsv', delimiter = '\t')

# Select Genome/Lineage columns and rename them
df_mg_g = df_mg_g[['Genome', 'Lineage', 'N50', 'Genome_type']]
df_mg_g = pd.merge(df_mg_g, df_mg_g_counts, on=['Genome'], how='left')
df_mg_g['Quality'] = df_mg_g.apply(compute_quality, axis=1)

df_mg_g = df_mg_g.rename(columns={'Genome': 'Genome_ID', 'Lineage': 'Taxon'})
df_mg_g['Taxon'] = df_mg_g['Taxon'].apply(format_taxonomy)

# Create Body_site and Specimen_type columns
df_mg_g['Body_site'] = 'Gut'
df_mg_g['Specimen_type'] = 'Gut_unclassified'
# Add Source column
df_mg_g['Source'] = 'EBI'
df_mg_g.drop(columns=['Count', 'Genome_type', 'N50'], inplace=True)

#### Parse MGnify oral

In [9]:
# Read MGnify oral metadata file
df_mg_o = pd.read_csv('files/input/mgnify_meta/genomes-all_metadata_oral.tsv', delimiter='\t')
df_mg_o_counts = pd.read_csv('files/output/mgnify_meta/mgnify_oral_fasta_counts.tsv', delimiter = '\t')

df_mg_o = df_mg_o[['Genome', 'Lineage', 'N50', 'Genome_type']]
df_mg_o = pd.merge(df_mg_o, df_mg_o_counts, on=['Genome'], how='left')

df_mg_o['Quality'] = df_mg_o.apply(compute_quality, axis=1)

df_mg_o = df_mg_o.rename(columns={'Genome': 'Genome_ID', 'Lineage': 'Taxon'})
df_mg_o['Taxon'] = df_mg_o['Taxon'].apply(format_taxonomy)

# Create Body_site and Specimen_type columns
df_mg_o['Body_site'] = 'Oral'
df_mg_o['Specimen_type'] = 'Oral_unclassified'
# Add Source column
df_mg_o['Source'] = 'EBI'
df_mg_o.drop(columns=['Count', 'Genome_type', 'N50'], inplace=True)

#### Add MGnify sample classification

In [10]:
# Modify sample classifications for MGnify data (final results taken from MGnify_meta_NEW.ipynb)
sample_gut = pd.read_csv('files/output/mgnify_biosample_meta/mgnify_meta_sample_gut.tsv', sep='\t')
sample_gut.rename(columns={'Genome': 'Genome_ID'}, inplace=True)

sample_oral = pd.read_csv('files/output/mgnify_biosample_meta/mgnify_meta_sample_oral.tsv', sep='\t')
sample_oral.rename(columns={'Genome': 'Genome_ID'}, inplace=True)

In [11]:
mgnify_gut = pd.merge(df_mg_g, sample_gut[['Genome_ID', 'Specimen']], on='Genome_ID', how='left')
mgnify_gut['Specimen'].fillna(mgnify_gut['Specimen_type'], inplace=True)
mgnify_gut.drop(columns = ['Body_site', 'Specimen_type'], axis=1, inplace=True)

mgnify_oral = pd.merge(df_mg_o, sample_oral[['Genome_ID', 'Specimen']], on='Genome_ID', how='left')
mgnify_oral['Specimen'].fillna(mgnify_oral['Specimen_type'], inplace=True)
mgnify_oral.drop(columns = ['Body_site', 'Specimen_type'], axis=1, inplace=True)

### HMP

#### Parse HMP

In [12]:
# Read HMP metadata file
df_hmp = pd.read_csv('files/input/hmp_meta/wgs_assembled_seq_set.tsv', delimiter='\t')

# Select id/body_site columns and rename them
df_hmp = df_hmp[['id', 'body_site']]
df_hmp = df_hmp.rename(columns={'id': 'Genome_ID', 'body_site': 'original_site'})
df_hmp['original_site'] = df_hmp['original_site'].str.lower()

df_hmp['Source'] = 'HMP'
df_hmp['Quality'] = 'Metagenome'

#### Add HMP sample classification

In [13]:
df_hmp['Specimen'] = pd.NA

# Assign Specimen type and category (body site) from classification table to HMP data
for index, row in df_hmp.iterrows():
    original_site = row['original_site']
    
    for _, class_row in df_class.iterrows():
        original_hmp = class_row['Original_HMP']
        # Check if 'original_site' from HMP table is contained in the 'Original_HMP' list of classification table
        if isinstance(original_hmp, list) and original_site in original_hmp:
            df_hmp.at[index, 'Specimen'] = class_row['Specimen']

# Remove original body site column
df_hmp = df_hmp.drop('original_site', axis=1)
df_hmp['Taxon'] = None

In [14]:
df_hmp.head()

,Genome_ID,Source,Quality,Specimen,Taxon
0,54a24ca84a57a7d5b06687939f211398,HMP,Metagenome,Urogenital_vagina,None
1,54a24ca84a57a7d5b06687939f48f9f4,HMP,Metagenome,Oral_cavity,None
2,54a24ca84a57a7d5b06687939f386ad9,HMP,Metagenome,Skin_head,None
3,54a24ca84a57a7d5b06687939f2fa7f6,HMP,Metagenome,Oral_mucosa,None
4,54a24ca84a57a7d5b06687939f38fda1,HMP,Metagenome,Oral_mucosa,None


### HMP reference genomes

#### Parse Bioproject PRJNA28331 metadata - Human Microbiome Project (HMP) Reference Genomes

In [16]:
# function for expanding rows with more than one GCF/GCA pair
def expand_rows(row):
    gcf_values = row['GCF_assembly_id']
    gca_values = row['GCA_assembly_id']
    num_values = max(len(gcf_values), len(gca_values))

    rows = []
    for i in range(num_values):
        row_data = {
            'biosample_id': row['biosample_id'],
            'bioproject_id': row['bioproject_id'],
            'isolation_source': row['isolation_source'],
            'misc_param_hmp_supersite': row['misc_param_hmp_supersite'],
            'misc_param_hmp_body_site': row['misc_param_hmp_body_site'],
            'tissue': row['tissue']}

        if i < len(gcf_values):
            row_data['GCF_assembly_id'] = gcf_values[i]
        else:
            row_data['GCF_assembly_id'] = None

        if i < len(gca_values):
            row_data['GCA_assembly_id'] = gca_values[i]
        else:
            row_data['GCA_assembly_id'] = None

        rows.append(row_data)

    return pd.DataFrame(rows)

with open('files/input/hmp_reference_meta/main_PRJNA28331.json') as file:
    main_project = json.load(file)

# Parse project metadata
all = set()
assembly_ids = []
level = []
biosample_ids = []
bioproject_ids = []
isolation_sources = []
misc_param_hmp_supersites = []
misc_param_hmp_body_sites = []
tissue_values = []
taxid = []
for entry in main_project["reports"]:   #fields of interest: 'assembly_level', 'bioproject_accession', 'biosample'
        accession = entry['accession']
        assembly_ids.append(accession)
        assembly_level = entry['assembly_info']['assembly_level']
        level.append(assembly_level)
        biosample_ids.append(entry['assembly_info']['biosample']['accession'])
        for projects in entry['assembly_info']['bioproject_lineage']:
            for bioprojects in projects['bioprojects']:
                if 'parent_accessions' in bioprojects and 'PRJNA28331' in bioprojects['parent_accessions']:
                    bioproject_ids.append(bioprojects['accession'])

        for attributes_key, attributes_value in entry['assembly_info']['biosample'].items():
            if attributes_key == 'attributes':
                transformed_dict = {}
                for item in attributes_value:
                    key = item['name']
                    value = item['value']
                    transformed_dict[key] = value

                isolation_sources.append(transformed_dict.get('isolation_source'))
                misc_param_hmp_supersites.append(transformed_dict.get('misc_param: HMP supersite'))
                misc_param_hmp_body_sites.append(transformed_dict.get('misc_param: HMP body site'))
                tissue_values.append(transformed_dict.get('tissue'))
                taxid.append(transformed_dict.get('host_taxid'))

data = {
'assembly_id': assembly_ids,
'level': level,
'biosample_id': biosample_ids,
'bioproject_id': bioproject_ids,
'isolation_source': isolation_sources,
'misc_param_hmp_supersite': misc_param_hmp_supersites,
'misc_param_hmp_body_site': misc_param_hmp_body_sites,
'tissue': tissue_values
}

df = pd.DataFrame(data)
df.drop_duplicates(inplace=True)
df['isolation_source'] = df['isolation_source'].replace('missing', None)
df['misc_param_hmp_body_site'] = df['misc_param_hmp_body_site'].replace('not determined', None)
df['misc_param_hmp_supersite'] = df['misc_param_hmp_supersite'].replace('not determined', None)
df['misc_param_hmp_supersite'] = df['misc_param_hmp_supersite'].replace('other', None)
pd.set_option('display.max_rows', 7000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)

df_grouped = df.groupby(['biosample_id', 'bioproject_id']).agg({'assembly_id': list}).reset_index()
df_grouped['GCF_assembly_id'] = df_grouped['assembly_id'].apply(lambda x: [a for a in x if a.startswith('GCF')]).apply(tuple)
df_grouped['GCA_assembly_id'] = df_grouped['assembly_id'].apply(lambda x: [a for a in x if a.startswith('GCA')]).apply(tuple)
df_grouped.drop(columns=['assembly_id'], inplace=True)

bioproject_df = df_grouped.merge(df[['biosample_id', 'bioproject_id', 'isolation_source', 'misc_param_hmp_body_site', 'misc_param_hmp_supersite', 'tissue']],
                                 on=['biosample_id', 'bioproject_id'], how='left')
bioproject_df.drop_duplicates(inplace=True)
bioproject_df.reset_index(inplace=True)
bioproject_df.drop(columns=['index'], inplace=True)
expanded_df = bioproject_df.apply(expand_rows, axis=1)
bioproject_df_final = pd.concat(expanded_df.to_list(), ignore_index=True)

# to remove the 'tissue' column, modify 1 row (since only 1 entry has values in this column), moving 'tissue'
# value to the 'misc_param_hmp_supersite' column
bioproject_df_final.loc[bioproject_df_final['GCF_assembly_id'] == 'GCF_000477535.1', 'misc_param_hmp_supersite'] = 'oral'
bioproject_df_final = bioproject_df_final.merge(df[['assembly_id', 'level']], left_on='GCA_assembly_id',
                                                right_on='assembly_id',
                                                how='left')
bioproject_df_final.drop(columns=['tissue', 'assembly_id'], inplace=True)
bioproject_df_final = bioproject_df_final[['GCA_assembly_id', 'GCF_assembly_id', 'level',
                                           'biosample_id', 'bioproject_id', 'isolation_source',
                                           'misc_param_hmp_body_site', 'misc_param_hmp_supersite']]


# Get a dataframe with rows containing no annotation at all
unannotated_df = bioproject_df_final[pd.isnull(bioproject_df_final['isolation_source']) &
                  pd.isnull(bioproject_df_final['misc_param_hmp_body_site']) &
                  pd.isnull(bioproject_df_final['misc_param_hmp_supersite'])]


# Filter the bioproject df to remove rows containing no annotation at all
bioproject_filtered = bioproject_df_final[~(
    pd.isnull(bioproject_df_final['isolation_source']) &
    pd.isnull(bioproject_df_final['misc_param_hmp_body_site']) &
    pd.isnull(bioproject_df_final['misc_param_hmp_supersite']))]

bioproject_filtered.reset_index(inplace=True)
bioproject_filtered.drop(columns=['index'], inplace=True)
bioproject_filtered['isolation_source'] = bioproject_filtered['isolation_source'].str.replace('"', '')

/tmp/ipykernel_39119/1479104618.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioproject_filtered.drop(columns=['index'], inplace=True)
/tmp/ipykernel_39119/1479104618.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bioproject_filtered['isolation_source'] = bioproject_filtered['isolation_source'].str.replace('"', '')


#### Add HMP reference sample classification

In [17]:
isolation_class = pd.read_csv('files/input/specimen_classifications/classifications_isolation_source.tsv', sep='\t')
hmp_body_class = pd.read_csv('files/input/specimen_classifications/classifications_hmp_bodysite.tsv', sep='\t')
hmp_super_class = pd.read_csv('files/input/specimen_classifications/classifications_hmp_supersite.tsv', sep='\t')


condition1 = (pd.notna(bioproject_filtered['isolation_source']))

condition2 = (pd.isnull(bioproject_filtered['isolation_source']) &
             pd.notna(bioproject_filtered['misc_param_hmp_body_site']))

condition3 = (pd.isnull(bioproject_filtered['isolation_source']) &
             pd.isnull(bioproject_filtered['misc_param_hmp_body_site']) &
             pd.notna(bioproject_filtered['misc_param_hmp_supersite']))

result_1 = bioproject_filtered[condition1].merge(isolation_class, on='isolation_source', how='left')
result_2 = bioproject_filtered[condition2].merge(hmp_body_class, on='misc_param_hmp_body_site', how='left')
result_3 = bioproject_filtered[condition3].merge(hmp_super_class, on='misc_param_hmp_supersite', how='left')

final_result = pd.concat([result_1, result_2, result_3], ignore_index=True)

# Manually replace annotations for PRJNA41955, PRJNA51495, PRJNA31017
final_result.loc[final_result['bioproject_id'] == 'PRJNA41955', 'Specimen'] = 'Gut_colon'
final_result.loc[final_result['bioproject_id'] == 'PRJNA51495', 'Specimen'] = 'Urogenital_vagina'
final_result.loc[final_result['bioproject_id'] == 'PRJNA51495', 'Body_site'] = 'Urogenital'
final_result.loc[final_result['bioproject_id'] == 'PRJNA31017', 'Specimen'] = 'Oral_plaque'

# Filter based on Healthy/Disease sample status
last_condition = final_result[final_result['Status'] != 'Disease']
hmp_ref_samples = last_condition[(last_condition['Body_site'] != 'Unclassified')]

#### Add HMP reference taxonomy

In [18]:
# Taxonomic lineage is taken from the latest version of GTDB (2023-06-09)
gtdb_bac = pd.read_csv('files/input/hmp_reference_meta/bac120_taxonomy.tsv', sep='\t', names=['assembly_ID', 'Taxonomy'])
gtdb_ar = pd.read_csv('files/input/hmp_reference_meta/ar53_taxonomy.tsv', sep='\t', names=['assembly_ID', 'Taxonomy'])
gtdb_taxa = pd.concat([gtdb_bac, gtdb_ar])

gtdb_taxa['assembly_ID'] = gtdb_taxa['assembly_ID'].str.replace(r'.*?(GCF|GCA)_', r'\1_', regex=True)

# Merge based on 'GCF_assembly_id', then on 'GCA_assembly_id' if 'Taxonomy' is NA
merged_df1 = hmp_ref_samples.merge(gtdb_taxa, left_on='GCF_assembly_id', right_on='assembly_ID', how='left')
missing_taxonomy = merged_df1[pd.isnull(merged_df1['Taxonomy'])]
merged_df1 = merged_df1[pd.notna(merged_df1['Taxonomy'])]


merged_df2 = missing_taxonomy.merge(gtdb_taxa, left_on='GCA_assembly_id', right_on='assembly_ID', how='left')
merged_df2.drop(columns=['assembly_ID_x', 'Taxonomy_x'], inplace=True)


bioproject_taxonomy = pd.concat([merged_df1, merged_df2], ignore_index=True)
bioproject_taxonomy.drop(columns=['assembly_ID', 'assembly_ID_y'], inplace=True)

bioproject_taxonomy['Taxonomy_z'] = bioproject_taxonomy['Taxonomy'].combine_first(bioproject_taxonomy['Taxonomy_y'])
bioproject_taxonomy.drop(columns=['Taxonomy', 'Taxonomy_y'], inplace=True)
bioproject_taxonomy = bioproject_taxonomy.rename(columns={'Taxonomy_z': 'Taxonomy'})


# Manually add taxonomy annotations for 6 samples
taxonomy_dict = {
    'GCA_000401635.1': 'd__Eukaryota;p__Mucoromycota;c__Mucoromycetes;o__Mucorales;f__Mucoraceae;g__Mucor;s__Mucor circinelloides',
    'GCA_000758865.1': 'd__Bacteria; p__Firmicutes; c__Tissierellia; o__; f__; g__; s__',
    'GCA_000759145.1': 'd__Bacteria; p__Firmicutes; c__Tissierellia; o__Tissierellales; f__Peptoniphilaceae; g__Peptoniphilus; s__Peptoniphilus lacrimalis',
    'GCA_001807715.1': 'd__Bacteria; p__Actinobacteria; c__Actinomycetia; o__Micrococcales; f__Micrococcaceae; g__Micrococcus; s__',
    'GCA_001546155.1': 'd__Bacteria; p__Firmicutes; c__Tissierellia; o__Tissierellales; f__Peptoniphilaceae; g__Anaerococcus; s__Anaerococcus hydrogenalis',
    'GCA_000220825.1': 'd__Bacteria; p__Fusobacteria; c__Fusobacteriia; o__Fusobacteriales; f__Fusobacteriaceae; g__Fusobacterium; s__Fusobacterium nucleatum'
}

for gca_id, taxonomy in taxonomy_dict.items():
    bioproject_taxonomy.loc[bioproject_taxonomy['GCA_assembly_id'] == gca_id, 'Taxonomy'] = taxonomy

    
bioproject_taxonomy['Taxonomy'] = bioproject_taxonomy['Taxonomy'].apply(format_taxonomy)

# keys_to_extract = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

# # Parse the lineage into separate columns
# for key in keys_to_extract:
#     bioproject_taxonomy[key] = bioproject_taxonomy['Taxonomy'].apply(lambda x: x.get(key, None))

# bioproject_taxonomy.drop(columns=['Taxonomy'], inplace=True)

bioproject_taxonomy = bioproject_taxonomy.rename(columns={'level': 'Quality', 'Taxonomy': 'Taxon'})


# Add Quality column, add Source column, perform final filtering
bioproject_taxonomy['Source'] = 'NCBI_PRJNA28331'
bioproject_taxonomy['Quality'] = bioproject_taxonomy['Quality'].replace({'Chromosome': 'Isolate (complete)', 'Complete Genome': 'Isolate (complete)',
                                              'Contig': 'Isolate (fragmented)', 'Scaffold': 'Isolate (fragmented)'})


bioproject_taxa_filtered = bioproject_taxonomy[~bioproject_taxonomy['Body_site'].str.contains('Joint|Eye', case=False, na=False)]

In [19]:
hmp_reference = bioproject_taxa_filtered[['GCA_assembly_id', 'Quality', 'Taxon', 'Source', 'Specimen']]
hmp_reference['Genome_ID'] = hmp_reference['GCA_assembly_id'].str.split('.').str[0]
hmp_ref_gc = pd.merge(hmp_reference, df_gc, on='Genome_ID', how='left')
hmp_ref_gc.drop(columns=['Genome_ID'], inplace=True)
hmp_ref_gc = hmp_ref_gc.rename(columns={'GCA_assembly_id': 'Genome_ID'})
hmp_ref_gc.head()

/tmp/ipykernel_39119/91954205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hmp_reference['Genome_ID'] = hmp_reference['GCA_assembly_id'].str.split('.').str[0]


,Genome_ID,Quality,Taxon,Source,Specimen,GC%
0,GCA_000174135.1,Isolate (fragmented),"{'Domain': 'Bacteria', 'Phylum': 'Bacillota', ...",NCBI_PRJNA28331,Skin_arm,32.769365
1,GCA_000174175.1,Isolate (fragmented),"{'Domain': 'Bacteria', 'Phylum': 'Campylobacte...",NCBI_PRJNA28331,Oral_plaque,44.849322
2,GCA_000159235.2,Isolate (fragmented),"{'Domain': 'Bacteria', 'Phylum': 'Actinomyceto...",NCBI_PRJNA28331,Urogenital_vagina,42.656827
3,GCA_000159375.2,Isolate (fragmented),"{'Domain': 'Bacteria', 'Phylum': 'Bacillota', ...",NCBI_PRJNA28331,Gut_unclassified,43.736761
4,GCA_000164155.2,Isolate (complete),"{'Domain': 'Bacteria', 'Phylum': 'Actinomyceto...",NCBI_PRJNA28331,Skin_unclassified,68.765442


### JGI

#### Parse JGI

In [20]:
# Read JGI metadata file
df_jgi = pd.read_csv('files/input/jgi_meta/genome_metadata_human.tsv', delimiter='\t')

# Select genome_id, ecosystem, ecosystem_type, habitat columns and rename them
df_jgi = df_jgi[['genome_id', 'ecosystem', 'ecosystem_type', 'habitat']]
df_jgi['original_site'] = df_jgi.apply(lambda row: f"{row['ecosystem_type']}/{row['habitat']}", axis=1)
df_jgi = df_jgi.drop('ecosystem_type', axis=1)
df_jgi = df_jgi.drop('habitat', axis=1)

df_jgi = df_jgi.rename(columns={'genome_id': 'Genome_ID', 'ecosystem': 'Taxon'})
df_jgi['Taxon'] = df_jgi['Taxon'].apply(format_taxonomy)
df_jgi['original_site'] = df_jgi['original_site'].str.lower()

df_jgi['Source'] = 'JGI'
df_jgi['Quality'] = 'MAG'

#### Add JGI sample classification

In [21]:
df_jgi['Specimen'] = pd.NA

# Assign Specimen type and category (body site) from classification table to JGI data
for index, row in df_jgi.iterrows():
    original_site = row['original_site']
    

    for _, class_row in df_class.iterrows():
        original_jgi = class_row['Original_JGI']
         # Check if 'original_site' from JGI table is contained in the 'Original_JGI' list of classification table
        if isinstance(original_jgi, list) and original_site in original_jgi:
            df_jgi.at[index, 'Specimen'] = class_row['Specimen']
# Remove original body site column
df_jgi = df_jgi.drop('original_site', axis=1)

In [22]:
df_jgi.head()

,Genome_ID,Taxon,Source,Quality,Specimen
0,3300007803_15,"{'Domain': 'Bacteria', 'Phylum': 'Firmicutes_C...",JGI,MAG,Gut_unclassified
1,3300007803_19,"{'Domain': 'Bacteria', 'Phylum': 'Campylobacte...",JGI,MAG,Gut_unclassified
2,3300007803_10,"{'Domain': 'Bacteria', 'Phylum': 'Bacteroidota...",JGI,MAG,Gut_unclassified
3,3300007803_26,"{'Domain': 'Bacteria', 'Phylum': 'Patescibacte...",JGI,MAG,Gut_unclassified
4,3300007803_9,"{'Domain': 'Bacteria', 'Phylum': 'Firmicutes',...",JGI,MAG,Gut_unclassified


## Merge all dataframes to obtain a full metadata table

In [23]:
# merged_hmp_jgi = pd.concat([df_hmp, df_jgi], axis=0)
merged_ebi_hmp_jgi = pd.concat([mgnify_gut, mgnify_oral, df_hmp, df_jgi], axis=0)

# Add GC content
metadata_table = pd.merge(merged_ebi_hmp_jgi, df_gc, on='Genome_ID', how='left')

final_df = pd.concat([metadata_table, hmp_ref_gc], axis=0)

In [24]:
keys_to_extract = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
for key in keys_to_extract:
    final_df[key] = final_df['Taxon'].apply(lambda x: x.get(key) if pd.notna(x) else None)

final_df.drop(columns=['Taxon'], inplace=True)

In [25]:
final_df = final_df.fillna('NA')

In [26]:
final_df.drop_duplicates(inplace=True)

In [28]:
final_df.to_csv('files/output/ABC_HuMi_all_metadata_latest.tsv', sep='\t', index=False)